# AOC2019 - Day 20

Today's challenge: [Day 20](https://adventofcode.com/2019/day/20)

Data are manually downloaded into `./data/input_20.txt`

In [1]:
from string import ascii_uppercase
from collections import defaultdict
import networkx as nx

In [2]:
with open('./data/input_20.txt') as f:
    raw = f.read()[:-1]
rows = raw.split("\n")

In [3]:
def nbs(x,y,all_=False):
    res = [(x+1,y),(x,y+1)]
    if all_:
        res += [(x-1,y),(x,y-1)]
    return res

labels = defaultdict(list)
base_G = nx.Graph()
for y, row in enumerate(rows):
    for x, c in enumerate(row):
        if c != '.':
            continue
        for x_, y_ in nbs(x, y,True):
            c_ = rows[y_][x_]
            if c_ == '.':
                base_G.add_edge((x,y),(x_,y_))
            elif c_ in ascii_uppercase:
                dx, dy = x_-x, y_-y
                next_c = rows[y_+dy][x_+dx]
                label = c_+next_c if dx+dy > 0 else next_c+c_
                labels[label].append((x,y))

G = nx.Graph(base_G.edges())
for _, nodes in labels.items():
    if len(nodes) == 2:
        G.add_edge(*nodes)

In [4]:
nx.shortest_path_length(G, labels['AA'][0],labels['ZZ'][0])

556

In [5]:
## Part 2 fractal maze!

In [6]:
W, H = len(rows[0]), len(rows)

def in_out(ps):
    x, y = ps[0]
    if x == 2 or x == W-3 or y == 2 or y == H-3:
        return ps[::-1]
    else:
        return ps

def get_level_edges(g, n):
    return [((*n1,n),(*n2,n)) for n1,n2 in g.edges()]

def add_level(g, n):
    g.add_edges_from(get_level_edges(base_G, n))
    for _, nodes in labels.items():
        if len(nodes) == 2:
            inner, outer = in_out(nodes)
            g.add_edge((*inner, n-1), (*outer, n))

In [7]:
full_G = nx.Graph(get_level_edges(base_G, 0))

start = (*labels['AA'][0], 0)
end = (*labels['ZZ'][0], 0)

n = 0
while True:
    n += 1
    add_level(full_G, n)
    if nx.has_path(full_G, start, end):
        break

print("layers:", n)
nx.shortest_path_length(full_G, start, end)

layers: 25


6532